# Developing a Model to Analyze Squad Battles Selection

## Environment Setup

In [3]:
%%capture
%pip install huggingface_hub numpy python-dotenv ultralytics

## Common Utilities

This section includes utility functions that are common across various parts of the notebook, aimed at streamlining workflow and simplifying code reuse.

### Additional helper functions can be defined below as needed

In [1]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file located one directory above the current notebook directory
load_dotenv(os.path.abspath(os.path.join('..', '.env')))  # Use relative path

# Retrieve the Hugging Face username and token from the environment
hf_username = os.getenv("HF_USERNAME")
hf_token = os.getenv("HF_TOKEN")

## Training
Train a model to classify available options in the squad battles selection menu.  

In [ ]:
from ultralytics import YOLO
import os

# Load YOLOv8 model
model = YOLO("yolov8n.pt")  # Use YOLOv8n, v8s, etc., depending on your needs

# Define the dataset path
dataset_path = os.path.expanduser("~/Downloads/project-3-at-2024-12-29-06-36-a6a284dc/dataset.yaml")  # Use os.path.expanduser to handle '~'

# Train model
model.train(
    data=dataset_path,  # Updated path to the dataset YAML
    imgsz=640,
    epochs=50,
    batch=16,
    name="yolo_training",
)

# Inference of the Model

In [ ]:
# Required imports
import json
import numpy as np
from ultralytics import YOLO
from huggingface_hub import hf_hub_download
from PIL import Image
import sys
import os

# Adding the directory containing 'src' to the system path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

# Load src.config to check the variables
import src.config as config

# Load the trained YOLO model
model_path = hf_hub_download(config.HF_SQUAD_SELECTION_PATH, config.SQUAD_SELECTION_FILENAME)
model = YOLO(model_path)

# Directory containing the images for inference
source_dir = os.path.expanduser("../screenshots/jpg/cropped/sample")  # Expand ~ to full path
output_dir = "runs/detect"  # Define output directory for annotations
output_json_path = os.path.join(output_dir, "annotations.json")  # Path for JSON annotations

# Ensure the output directory exists
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Initialize list for annotations
annotations = []

# Base URL of the locally hosted server
base_url = "http://127.0.0.1:8085/"

# Counter for generating unique numeric IDs
image_id_counter = 1
result_id_counter = 1

# Iterate through all images in the directory
for filename in os.listdir(source_dir):
    if filename.endswith(".jpg"):
        image_path = os.path.join(source_dir, filename)
        image = Image.open(image_path).convert("RGB")

        # Convert PIL image to NumPy array
        image_np = np.array(image)

        # Send the image to the YOLO model for detection without saving images
        model_result = model.predict(source=image_np, imgsz=640, conf=0.25, save=False)  # Set save=False

        # Get the bounding boxes, confidence, and class IDs
        boxes = model_result[0].boxes.xyxy.cpu().numpy()  # (x1, y1, x2, y2)
        conf = model_result[0].boxes.conf.cpu().numpy()
        cls = model_result[0].boxes.cls.cpu().numpy().astype(int)

        # Initialize the overall annotation structure for the current image
        image_annotations = {
            "id": image_id_counter,  # Use a counter for the ID of the image entry
            "data": {
                "image": f"{base_url}{filename}",  
            },
            "annotations": [
                {
                "id": 1,  
                "result": []
                }
            ]
        }

        # Populate the annotations list to match the specified format
        for box, confidence, class_id in zip(boxes, conf, cls):
            x1, y1, x2, y2 = box.astype(float)
            width = x2 - x1
            height = y2 - y1
            if confidence > config.SQUAD_SELECTION_CONF_THRESHOLD:
                result = {
                    "original_width": image.width,
                    "original_height": image.height,
                    "image_rotation": 0,
                    "value": {
                        "x": x1 / image.width * 100,  # Calculate relative x
                        "y": y1 / image.height * 100,  # Calculate relative y
                        "width": width / image.width * 100,  # Width in percentage
                        "height": height / image.height * 100,  # Height in percentage
                        "rotation": 0,
                        "rectanglelabels": [model_result[0].names[class_id]]  # Replace with actual class names
                    },
                    "id": result_id_counter,  # Use the same counter
                    "from_name": "label",
                    "to_name": "image",
                    "type": "rectanglelabels",
                    "origin": "manual"
                }
                image_annotations["annotations"][0]["result"].append(result)  # Fix applied here
                result_id_counter += 1  # Increment the annotation ID for the next annotation

        # Append image annotations to the master list
        annotations.append(image_annotations)
        image_id_counter += 1  # Increment the image ID for the next image

# Save annotations to JSON file
with open(output_json_path, 'w') as json_file:
    json.dump(annotations, json_file, indent=4)

print(f"Annotations saved to {output_json_path}.")

## Host HTTP Server Locally for Image Files
To verify the run using Label Studio, host the image files so that the annotations.json can be imported properly.

In [14]:
%%capture
%pip install Flask Flask-CORS

In [ ]:
from flask import Flask, send_from_directory
from flask_cors import CORS
import threading
import os

app = Flask(__name__)
CORS(app)  # Enable CORS for all routes

# Assuming source_dir is defined in an earlier cell
if source_dir:
    directory_to_serve = source_dir
else:
    # Otherwise just host the Desktop
    directory_to_serve = os.path.expanduser("~/Desktop/results")

@app.route('/<path:filename>', methods=['GET'])
def serve_image(filename):
    return send_from_directory(directory_to_serve, filename)

# Function to run the server
def run_server():
    app.run(host='0.0.0.0', port=8085)  # Bind to all interfaces

# Start the server in a new thread
threading.Thread(target=run_server).start()

# Upload to HuggingFace

In [ ]:
from huggingface_hub import HfApi, Repository
import os
import shutil

# Set your Hugging Face token and repository name
repo_name = "fc24-squad-battles-selection"  # Replace with the desired repository name

# Define paths
model_dir = "./runs/detect/yolo_training" 
local_repo_dir = f"huggingface_yolo_repo"  # Temporary local directory for the repository

# Create a new repository on Hugging Face (or use an existing one)
api = HfApi()
repo_url = api.create_repo(repo_id=repo_name, token=hf_token, exist_ok=True)

# Add user information
git_user = "bendythepirate"        # Replace with your Hugging Face username
git_email = "bendy@bendythepirate.com"  # Replace with your email associated with Hugging Face

# Clone the repository to a local folder with the token
repo = Repository(
    local_dir=local_repo_dir,
    clone_from=repo_url,
    use_auth_token=hf_token,
    git_user=git_user,
    git_email=git_email
)

# Copy all files from the training directory to the repository
shutil.copytree(model_dir, os.path.join(local_repo_dir, "weights"), dirs_exist_ok=True)

# Push the files to Hugging Face
repo.push_to_hub(commit_message="Upload YOLO training output")

print(f"Files uploaded successfully! View the repository here: {repo_url}")
